In [1]:
!python src/WikiSplitter.py split-wiki dump.xml


Path to wiki xml file: ^C
Aborted!


In [ ]:
%pip install -U pip setuptools wheel
%pip install -U spacy


In [ ]:
!python -m spacy download en_core_web_sm


In [1]:
import spacy
import json

nlp = spacy.load("en_core_web_sm")


In [ ]:
!node parseWikitext.js


In [8]:
import aiohttp
import asyncio
import json
import os


In [10]:
async def fetch_pageviews(session, article, year):
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article"
    domain = "en.wikipedia.org"
    access = "all-access"
    agent = "user"

    start_date = f"{year}0101"
    end_date = f"{year}1231"

    url = f"{base_url}/{domain}/{access}/{agent}/{article}/monthly/{start_date}/{end_date}"

    async with session.get(url) as response:
        if response.status == 200:
            data = await response.json()
            return article, data
        else:
            return article, None


In [11]:
async def process_articles(articles_titles, year):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for article_title in articles_titles:
            task = asyncio.ensure_future(
                fetch_pageviews(session, article_title, year))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)

        for article, data in responses:
            if data:
                total_views = sum(item['views'] for item in data['items'])
                print(
                    f"Total Pageviews for {article} in {year}: {total_views}")
            else:
                print(f"Failed to retrieve data for {article}")


In [13]:
def load_articles_from_json(folder_path, processed_files, max_articles=500):
    articles = []
    for filename in os.listdir(folder_path):
        if len(articles) >= max_articles:
            return articles, processed_files
        if filename.endswith('.json') and filename not in processed_files:
            with open(os.path.join(folder_path, filename)) as f:
                articles = json.load(f)
                for article in articles:
                    articles.append(article)    
                processed_files.add(filename)
            processed_files.add(filename)
    return articles, processed_files


In [28]:
async def fetch_pageviews_for_articles(folder_path="./2_wikitext_parser_output/",  year="2023",    processed_files_path='./processed_files.txt', files_at_once=6):

    # Lade die Namen der bereits verarbeiteten Dateien
    if os.path.exists(processed_files_path):
        with open(processed_files_path, 'r') as f:
            processed_files = set(f.read().splitlines())
    else:
        processed_files = set()

    while True:
        articles, processed_files = load_articles_from_json(
            folder_path, processed_files, max_articles=200)
        print(len(articles))
        break
        if not articles:
            break  # Beendet die Schleife, wenn keine Artikel mehr zu verarbeiten sind

        await process_articles(articles, year)

        # Aktualisiere die Liste der verarbeiteten Dateien
        with open(processed_files_path, 'w') as f:
            for file in processed_files:
                f.write(file + '\n')


In [29]:
await fetch_pageviews_for_articles()


201


In [84]:
with open('parsed_articles.json') as file:
    parsed_articles = json.load(file)


In [85]:
def clean_str(string: str):
  return string.replace(
                '\n', ' ').replace('  ', ' ').strip()


In [86]:
articles = []
for article in parsed_articles:
    doc = nlp(article['history'])
    sentences = list(doc.sents)
    sentences_with_dates = []

    for ent in doc.ents:
        if ent.label_ == "DATE":
            current_sentence = ent.sent
            current_index = sentences.index(current_sentence)
            current_sentence_text = clean_str(current_sentence.text)

            # Hinzufügen des vorherigen Satzes, wenn vorhanden und noch nicht in der Liste
            # if current_index > 0 and sentences[current_index - 1].text not in sentences_with_dates:
                # sentences_with_dates.append(sentences[current_index - 1].text)

            # Hinzufügen des aktuellen Satzes, wenn noch nicht in der Liste
            if current_sentence_text not in sentences_with_dates:
                sentences_with_dates.append(current_sentence_text)

            # Hinzufügen des nachfolgenden Satzes, wenn vorhanden und noch nicht in der Liste
            # if current_index < len(sentences) - 1 and sentences[current_index + 1].text not in sentences_with_dates:
                # sentences_with_dates.append(sentences[current_index + 1].text)
        
    article['sentencesWithDates'] = ' '.join(sentences_with_dates)
    articles.append(article)


In [92]:
len(articles)


255

In [88]:
import re


def remove_first_parentheses_in_first_sentence(text):
    sentences = re.split(r'(?<=[.!?])\s+', text, maxsplit=1)
    first_sentence = sentences[0]
    remaining_text = sentences[1] if len(sentences) > 1 else ''
    cleaned_first_sentence = re.sub(r'\(.*?\)', '', first_sentence, count=1)
    return cleaned_first_sentence + ' ' + remaining_text


In [89]:
cleaned_articles = []
for article in articles:
    if article['sentencesWithDates'] == '':
        continue
    if 'shortDescription' in article:
        article['shortDescription'] = remove_first_parentheses_in_first_sentence(
            article['shortDescription'])
    if 'description' in article:
        article['description'] = remove_first_parentheses_in_first_sentence(
            article['description'])
    if article['sentencesWithDates'] == '':
        continue
    cleaned_articles.append(article)


In [93]:
import requests


def get_wikipedia_pageviews_2022(title):
    """
    Ermittelt die Gesamtzahl der Seitenaufrufe für einen Wikipedia-Artikel im Jahr 2022.
    
    :param title: Titel des Wikipedia-Artikels.
    :return: Anzahl der Seitenaufrufe oder eine Fehlermeldung.
    """
    # URL für die Wikipedia API, die Seitenaufrufe für das Jahr 2022 abruft
    url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/user/{title}/daily/2022010100/2022123100"

    try:
        headers = {
          'User-Agent': 'MeinPythonSkript/1.0 (meineemail@example.com)'
        }
        response = requests.get(url, headers=headers)

        response.raise_for_status()
        data = response.json()

        total_views = sum(item['views'] for item in data['items'])
        return total_views
    except requests.RequestException as e:
        return f"Fehler bei der Anfrage: {e}"
    except KeyError:
        return "Fehler bei der Verarbeitung der Daten."


In [94]:
cleaned_articles_with_clicks = []
for article in cleaned_articles:
    if 'title' in article:
        article['clicks'] = get_wikipedia_pageviews_2022(article['title'])
        print(f"Seitenaufrufe für {article['title']}: {article['clicks']}")
    else:
        article['clicks'] = None
        print(f"Keine Seitenaufrufe für diesen {article['title']} verfügbar.")
    cleaned_articles_with_clicks.append(article)


Seitenaufrufe für Alberta: 1020808
Seitenaufrufe für Ankara: 640512
Seitenaufrufe für Aberdeenshire: 198238
Seitenaufrufe für Azincourt: 41905
Seitenaufrufe für Achill Island: 102543
Seitenaufrufe für Anguilla: 529462
Seitenaufrufe für Ashmore and Cartier Islands: 72160
Seitenaufrufe für Ames, Iowa: 169544
Seitenaufrufe für Abensberg: 5275
Seitenaufrufe für Andes: 521063
Seitenaufrufe für Anazarbus: 7764
Seitenaufrufe für Anadyr (river): 3459
Seitenaufrufe für Ambracia: 9280
Seitenaufrufe für Abydos, Egypt: 95347
Seitenaufrufe für Aberdeen: 437607
Seitenaufrufe für Actium: 26777
Seitenaufrufe für Abadan, Iran: 38464
Seitenaufrufe für Adobe Inc.: 610329
Seitenaufrufe für Austin, Texas: 1397545
Seitenaufrufe für Crandall University: 13363
Seitenaufrufe für Acadia University: 35829
Seitenaufrufe für Annapolis Valley: 21087
Seitenaufrufe für AFC Ajax: 1923325
Seitenaufrufe für Area 51: 1727047
Seitenaufrufe für Albion, Michigan: 15618
Seitenaufrufe für Alicante: 294728
Seitenaufrufe für Ab

In [20]:
with open('articles_with_dates.json', 'w') as file:
    json.dump(cleaned_articles, file, ensure_ascii=False,)


In [ ]:
for article in articles:
    print(
        article['title'], len(article['history']), len(article['sentencesWithDates']))
    print('------------------')
